# Imports

In [3]:
import librosa

import numpy as np
import pandas as pd
import random

import torch
import torchmetrics
import os
import torch.nn.functional as F
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from joblib import Parallel, delayed

d:\Anaconda3\Lib\site-packages\pkg_resources\__init__.py:116: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2-msvc is an invalid version and will not be supported in a future release
  warnings.warn(
d:\Anaconda3\Lib\site-packages\pkg_resources\__init__.py:116: PkgResourcesDeprecationWarning: 0.996-ko-0.9.2-msvc is an invalid version and will not be supported in a future release
  warnings.warn(


In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

### Config

In [5]:
class Config:
    SR = 32000
    N_MFCC = 40
    
    # Dataset
    ROOT_DIR = 'C:/HongBeomsun/Dataset_SSD/FakeVoice'
    
    # Training
    N_CLASSES = 2
    BATCH_SIZE = 96
    N_EPOCHS = 100
    LEARNING_RATE = 1e-3
    
    # Others
    SEED = 42
    
CONFIG = Config()

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [7]:
seed_everything(CONFIG.SEED)

### Data

In [8]:
df = pd.read_csv(os.path.join(CONFIG.ROOT_DIR,'train.csv'))

In [9]:
print(len(df))
df.head()

55438


,id,path,label
0,RUNQPNJF,./train/RUNQPNJF.ogg,real
1,JFAWUOGJ,./train/JFAWUOGJ.ogg,fake
2,RDKEKEVX,./train/RDKEKEVX.ogg,real
3,QYHJDOFK,./train/QYHJDOFK.ogg,real
4,RSPQNHAO,./train/RSPQNHAO.ogg,real


In [10]:
df['label'].value_counts()

label
fake    27818
real    27620
Name: count, dtype: int64

In [11]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CONFIG.SEED, stratify=df['label'])

In [12]:
train['label'].value_counts()
val['label'].value_counts()

label
fake    5564
real    5524
Name: count, dtype: int64

### Pre-processing : MFCC & DeltaMFCC

In [15]:
def get_features(df, train_mode=True, use_parallel=False):
    features = []
    labels = []
    for i, (index, row) in enumerate(tqdm(df.iterrows(), total=len(df)), 1):
        try:
            y, sr = librosa.load(os.path.join(CONFIG.ROOT_DIR, row['path']), sr=CONFIG.SR)
            
            mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CONFIG.N_MFCC)
            delta_mfcc = librosa.feature.delta(mfcc)
            
            combined = np.vstack([mfcc, delta_mfcc])
            combined_mean = np.mean(combined, axis=1)
            features.append(combined_mean)
            
            if train_mode:
                label = row['label']
                label_vector = np.zeros(CONFIG.N_CLASSES, dtype=float)
                label_vector[0 if label == 'fake' else 1] = 1
                labels.append(label_vector)
                
        except Exception as e:
            print(f'Error while {index} : {e}')
            continue
    
    if train_mode:
        return features, labels
    return features
        

In [16]:
train_features, train_labels = get_features(train)
val_features, val_labels = get_features(val)

 59%|█████▉    | 26072/44350 [06:13<04:01, 75.75it/s]

Error while 26061 : when mode='interp', width=9 cannot exceed data.shape[axis]=7


 72%|███████▏  | 31955/44350 [07:36<02:43, 75.81it/s]

Error while 31944 : when mode='interp', width=9 cannot exceed data.shape[axis]=8


100%|██████████| 11088/11088 [02:14<00:00, 82.32it/s] 


In [17]:
print(train_features[10].shape)
print(len(train_features))
print(len(train_labels))

(80,)
44348
44348


In [18]:
def save_np():
    os.makedirs(os.path.join(CONFIG.ROOT_DIR, 'npy'), exist_ok=True)
    np.save(os.path.join(CONFIG.ROOT_DIR, 'npy/train_features_mean.npy'), train_features)
    np.save(os.path.join(CONFIG.ROOT_DIR, 'npy/train_labels_mean.npy'), train_labels)
    np.save(os.path.join(CONFIG.ROOT_DIR, 'npy/val_features_mean.npy'), val_features)
    np.save(os.path.join(CONFIG.ROOT_DIR, 'npy/val_labels_mean.npy'), val_labels)

In [19]:
def load_np():
    train_features = np.load(os.path.join(CONFIG.ROOT_DIR, 'npy/train_features.npy'))
    train_labels = np.load(os.path.join(CONFIG.ROOT_DIR, 'npy/train_labels.npy'))
    val_features = np.load(os.path.join(CONFIG.ROOT_DIR, 'npy/val_features.npy'))
    val_labels = np.load(os.path.join(CONFIG.ROOT_DIR, 'npy/val_labels.npy'))
    
    return train_features, train_labels, val_features, val_labels

In [20]:
save_np()
# train_features, train_labels, val_features, val_labels = load_np()

In [25]:
train_features = np.array(train_features)
train_labels = np.array(train_labels)
val_features = np.array(val_features)
val_labels = np.array(val_labels)

In [26]:
print(train_features.shape, len(train_labels))
print(val_features.shape, len(val_labels))

(44348, 80) 44348
(11088, 80) 11088


In [27]:
print(train_features[0], train_labels[0])

[-3.6298355e+02  1.2409643e+02 -5.4747303e+01  7.0447746e+01
 -2.6750314e+00  1.0377989e+01  2.1588305e+01 -1.1392265e+01
  1.5144799e+01 -6.1271892e+00  9.7842264e+00  6.8455625e+00
 -2.6896639e+00  1.3830722e+01 -5.1628785e+00  9.7452707e+00
  7.0354652e+00  7.6968670e-02  1.1705402e+01 -7.1903372e-01
  8.0656118e+00  2.2053039e+00 -2.8435019e-01  7.1962514e+00
 -2.3920376e+00  5.1100283e+00 -5.1483619e-01  1.0345262e+00
  4.8815756e+00 -3.1192234e+00  6.9965553e+00  4.8622530e-02
  3.8208070e-01  6.5177312e+00 -6.4668083e-01  4.9323301e+00
  1.3422136e+00  1.8539882e+00  3.2438755e+00 -1.1100441e+00
 -1.5834183e-01  1.3949156e-02 -4.3554106e-03  3.9133374e-02
 -1.1917626e-02  4.1101277e-03  2.5846034e-02  2.5671145e-02
  2.9278807e-02 -2.4162741e-02 -1.5184934e-02 -3.6436853e-03
 -8.2941456e-03 -1.0004562e-02 -2.2768199e-02 -2.4706349e-03
  4.7766133e-03  5.7219812e-03  9.9832593e-03 -1.1366978e-02
 -7.5917068e-04 -1.4376579e-03 -6.7380914e-03  9.1720149e-03
 -3.3423454e-03 -2.46119

### Dataset

In [28]:
class CustomDataset(Dataset):
    def __init__(self, mfcc, label):
        self.mfcc = mfcc
        self.label = label

    def __len__(self):
        return len(self.mfcc)

    def __getitem__(self, index):
        if self.label is not None:
            return self.mfcc[index], self.label[index]
        return self.mfcc[index]

In [29]:
train_dataset = CustomDataset(train_features, train_labels)
val_dataset = CustomDataset(val_features, val_labels)

In [32]:
train_loader = DataLoader(
    train_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

### Define Model

In [33]:
class MLP(nn.Module):
    def __init__(self, input_dim=80, output_dim=CONFIG.N_CLASSES):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.dropout1 = nn.Dropout(0.3)
        
        self.fc2 = nn.Linear(256, 128)
        self.bn2 = nn.BatchNorm1d(128)
        self.dropout2 = nn.Dropout(0.3)
        
        self.fc3 = nn.Linear(128, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout1(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout2(x)
        x = torch.sigmoid(self.fc3(x))
        return x

### Train & Validation

In [34]:
from sklearn.metrics import roc_auc_score
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

In [35]:
def train(model, scheduler, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.BCELoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CONFIG.N_EPOCHS+1):
        model.train()
        train_loss = []
        for features, labels in tqdm(iter(train_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            optimizer.zero_grad()
            
            output = model(features)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val AUC : [{_val_score:.5f}] LEARNING RATE : [{optimizer.param_groups[0]["lr"]:.5f}]')

        scheduler.step(_val_loss)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
    
    return best_model

In [36]:
def multiLabel_AUC(y_true, y_scores):
    auc_scores = []
    for i in range(y_true.shape[1]):
        auc = roc_auc_score(y_true[:, i], y_scores[:, i])
        auc_scores.append(auc)
    mean_auc_score = np.mean(auc_scores)
    return mean_auc_score

In [37]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss, all_labels, all_probs = [], [], []
    
    with torch.no_grad():
        for features, labels in tqdm(iter(val_loader)):
            features = features.float().to(device)
            labels = labels.float().to(device)
            
            probs = model(features)
            
            loss = criterion(probs, labels)

            val_loss.append(loss.item())

            all_labels.append(labels.cpu().numpy())
            all_probs.append(probs.cpu().numpy())
        
        _val_loss = np.mean(val_loss)

        all_labels = np.concatenate(all_labels, axis=0)
        all_probs = np.concatenate(all_probs, axis=0)
        
        # Calculate AUC score
        auc_score = multiLabel_AUC(all_labels, all_probs)
    
    return _val_loss, auc_score

### Run

In [38]:
model = MLP()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CONFIG.LEARNING_RATE)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

infer_model = train(model, scheduler, optimizer, train_loader, val_loader, device)

100%|██████████| 116/116 [00:00<00:00, 555.33it/s]


Epoch [1], Train Loss : [0.14385] Val Loss : [0.03472] Val AUC : [0.99927] LEARNING RATE : [0.00100]


100%|██████████| 116/116 [00:00<00:00, 583.29it/s]


Epoch [2], Train Loss : [0.04731] Val Loss : [0.02429] Val AUC : [0.99951] LEARNING RATE : [0.00100]


100%|██████████| 116/116 [00:00<00:00, 620.19it/s]


...

Epoch [98], Train Loss : [0.00294] Val Loss : [0.00416] Val AUC : [0.99999] LEARNING RATE : [0.00000]


100%|██████████| 116/116 [00:00<00:00, 594.77it/s]


Epoch [99], Train Loss : [0.00340] Val Loss : [0.00382] Val AUC : [0.99999] LEARNING RATE : [0.00000]


100%|██████████| 116/116 [00:00<00:00, 604.85it/s]

Epoch [100], Train Loss : [0.00302] Val Loss : [0.00384] Val AUC : [0.99999] LEARNING RATE : [0.00000]


### Inference

In [39]:
def get_features_without_e(df, train_mode=True, use_parallel=False):
    features = []
    labels = []
    for i, (index, row) in enumerate(tqdm(df.iterrows(), total=len(df)), 1):
        y, sr = librosa.load(os.path.join(CONFIG.ROOT_DIR, row['path']), sr=CONFIG.SR)
        
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CONFIG.N_MFCC)
        delta_mfcc = librosa.feature.delta(mfcc)
        
        combined = np.vstack([mfcc, delta_mfcc])
        combined_mean = np.mean(combined, axis=1)
        features.append(combined_mean)
        
        if train_mode:
            label = row['label']
            label_vector = np.zeros(CONFIG.N_CLASSES, dtype=float)
            label_vector[0 if label == 'fake' else 1] = 1
            labels.append(label_vector)
    
    if train_mode:
        return features, labels
    return features

In [40]:
test = pd.read_csv(os.path.join(CONFIG.ROOT_DIR, 'test.csv'))
test_mfcc = get_features_without_e(test, False)
test_dataset = CustomDataset(test_mfcc, None)
test_loader = DataLoader(
    test_dataset,
    batch_size=CONFIG.BATCH_SIZE,
    shuffle=False
)

100%|██████████| 50000/50000 [15:33<00:00, 53.56it/s]


In [41]:
# np.save(os.path.join(CONFIG.ROOT_DIR, 'npy/test_mfcc.npy'), test_mfcc)
# test_mfcc = np.load(os.path.join(CONFIG.ROOT_DIR, 'npy/test_mfcc.npy'))

In [42]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    predictions = []
    with torch.no_grad():
        for features in tqdm(iter(test_loader)):
            features = features.float().to(device)
            
            probs = model(features)

            probs  = probs.cpu().detach().numpy()
            predictions += probs.tolist()
    return predictions

In [43]:
preds = inference(infer_model, test_loader, device)

100%|██████████| 521/521 [00:00<00:00, 774.58it/s]


## Submission

In [45]:
submit = pd.read_csv(os.path.join(CONFIG.ROOT_DIR,'./sample_submission.csv'))
submit.iloc[:, 1:] = preds
submit.head()

,id,fake,real
0,TEST_00000,9.944219e-01,0.005529
1,TEST_00001,9.809787e-01,0.018809
2,TEST_00002,9.812275e-01,0.018559
3,TEST_00003,1.145170e-04,0.999887
4,TEST_00004,8.947465e-07,0.999999


In [47]:
submit.to_csv(f'./output/submit_MLP_SR{CONFIG.SR}_N_MFCC{CONFIG.N_MFCC}_EPOCH{CONFIG.N_EPOCHS}_BATCHSIZE{CONFIG.BATCH_SIZE}.csv', index=False)